In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .appName("crear_tablas_hive") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2023-11-18T21:53:11,471 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-11-18T21:53:12,940 WARN [Thread-4] org.apache.spark.util.Utils - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2023-11-18T21:53:12,941 WARN [Thread-4] org.apache.spark.util.Utils - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
# Crear una nueva base de datos en Hive
spark.sql("CREATE DATABASE if not exists datalake")

# Listar bases de datos para verificar
spark.sql("SHOW DATABASES").show()

2023-11-18T21:59:24,672 INFO [Thread-4] org.apache.hadoop.hive.conf.HiveConf - Found configuration file file:/home/ort/spark/conf/hive-site.xml
2023-11-18T21:59:24,915 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.metastore.wm.default.pool.size does not exist
2023-11-18T21:59:24,915 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.llap.task.scheduler.preempt.independent does not exist
2023-11-18T21:59:24,916 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.llap.output.format.arrow does not exist
2023-11-18T21:59:24,916 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.tez.llap.min.reducer.per.executor does not exist
2023-11-18T21:59:24,916 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.arrow.root.allocator.limit does not exist
2023-11-18T21:59:24,916 WARN [Thread-4] org.apache.hadoop.hive.conf.HiveConf - HiveConf of name hive.vectorized.use.che

In [3]:
spark.sql("use datalake")

DataFrame[]

In [4]:
spark.sql("""
CREATE EXTERNAL TABLE usuarios (
    id_usuario STRING,
    nombre STRING,
    apellido STRING,
    email STRING,
    sexo  STRING,
    pais STRING,
    actividad STRING,
    subscripcion STRING,
    fecha_subs DATE,
    recha_renov DATE, 
    ultimo_pago DATE
)
STORED AS PARQUET
LOCATION '/datalake/clean/usuarios';
""")

2023-11-18T22:07:35,046 INFO [Thread-4] org.apache.hadoop.hive.ql.security.authorization.plugin.sqlstd.SQLStdHiveAccessController - Created SQLStdHiveAccessController for session context : HiveAuthzSessionContext [sessionString=349f44e8-f284-435b-b59b-f9608c9f8ff9, clientType=HIVECLI]
2023-11-18T22:07:35,051 WARN [Thread-4] org.apache.hadoop.hive.ql.session.SessionState - METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
2023-11-18T22:07:35,051 INFO [Thread-4] hive.metastore - Mestastore configuration hive.metastore.filter.hook changed from org.apache.hadoop.hive.metastore.DefaultMetaStoreFilterHookImpl to org.apache.hadoop.hive.ql.security.authorization.plugin.AuthorizationMetaStoreFilterHook
2023-11-18T22:07:35,058 INFO [Thread-4] hive.metastore - Closed a connection to metastore, current connections: 0
2023-11-18T22:07:35,060 INFO [Thread-4] hive.metastore - Trying to connect to metastore with URI thrift://l

DataFrame[]

In [7]:
df_actividades = spark.read.parquet("/datalake/clean/actividades")

In [8]:
df_actividades.printSchema()

root
 |-- id_actividad: string (nullable = true)
 |-- id_usuario: string (nullable = true)
 |-- id_dispositivo: string (nullable = true)
 |-- id_plan: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- longitud: float (nullable = true)
 |-- latitud: float (nullable = true)
 |-- duracion: integer (nullable = true)
 |-- fecha_hora: timestamp (nullable = true)



In [5]:
spark.sql("""
select *
from usuarios
""").show()

+--------------------+-----------+-------------+--------------------+----------+--------------+-------------+------------+----------+-----------+-----------+
|          id_usuario|     nombre|     apellido|               email|      sexo|          pais|    actividad|subscripcion|fecha_subs|recha_renov|ultimo_pago|
+--------------------+-----------+-------------+--------------------+----------+--------------+-------------+------------+----------+-----------+-----------+
|9efa1289-5256-49c...|      Elise|       Austen|  eausten1@unblog.fr|    Female|        Brazil|     natacion|      basica|2022-12-13|       null| 2022-12-25|
|b4d30551-59cb-45b...|   Rochella|      Davidof|rdavidof2@timeson...|Non-binary|        Russia|personalizado|      basica|2023-04-22|       null| 2023-07-20|
|effbcd55-e394-487...|Worthington|  Sherringham|wsherringham5@tut...|      Male|     Indonesia|personalizado|       anual|2023-03-26|       null| 2023-10-27|
|fc621f46-e0ef-4ad...|    Camella|       Crooke|ccro

In [14]:
spark.sql("""
CREATE EXTERNAL TABLE actividades (
    id_actividad STRING,
    id_usuario STRING,
    id_dispositivo STRING,
    id_plan STRING,
    pais STRING,
    longitud FLOAT,
    latitud FLOAT,
    duracion INT,
    fecha_hora TIMESTAMP
)
STORED AS PARQUET
LOCATION '/datalake/clean/actividades';
""")

DataFrame[]

In [15]:
spark.sql("select * from actividades").show()

+--------------------+--------------------+--------------------+--------------------+--------------+--------+--------+--------+-------------------+
|        id_actividad|          id_usuario|      id_dispositivo|             id_plan|          pais|longitud| latitud|duracion|         fecha_hora|
+--------------------+--------------------+--------------------+--------------------+--------------+--------+--------+--------+-------------------+
|d2b9f5a4-e457-406...|9efa1289-5256-49c...|5f6742cf-0cc3-499...|1a257404-2741-4d5...|        Brazil|-47.9247|-38.5258|       1|2023-07-08 06:10:00|
|742ceebe-7cbd-42a...|b4d30551-59cb-45b...|9c9d7ede-4915-4fd...|296f1010-fd8b-4fe...|        Russia|100.6432| 60.0272|       2|2023-03-01 02:16:00|
|f079c279-1249-40e...|effbcd55-e394-487...|af1e4b22-9836-4ff...|d3c2f38b-249f-40c...|     Indonesia|112.6225| -7.7611|       3|2023-09-17 07:09:00|
|b8db20aa-9915-426...|fc621f46-e0ef-4ad...|5d5aa38b-9e5e-412...|b206f2a3-5ddd-47a...|        Russia|100.6432| 60

In [16]:
df_actividades_eventos = spark.read.parquet("/datalake/clean/actividades_eventos")

In [17]:
df_actividades_eventos.printSchema()

root
 |-- id_sensor: string (nullable = true)
 |-- pulse_rate: integer (nullable = true)
 |-- id_actividad: string (nullable = true)
 |-- longitud: float (nullable = true)
 |-- latitud: float (nullable = true)
 |-- fecha_hora: timestamp (nullable = true)



In [19]:
spark.sql("""
CREATE EXTERNAL TABLE actividades_eventos (
    id_sensor STRING,
    pulse_rate integer,
    id_actividad STRING,
    longitud FLOAT,
    latitud FLOAT,
    fecha_hora TIMESTAMP
)
STORED AS PARQUET
LOCATION '/datalake/clean/actividades_eventos';
""")

DataFrame[]

In [20]:
spark.sql("select * from actividades_eventos").show()

+--------------------+----------+--------------------+--------+--------+-------------------+
|           id_sensor|pulse_rate|        id_actividad|longitud| latitud|         fecha_hora|
+--------------------+----------+--------------------+--------+--------+-------------------+
|853dc9e4-6094-451...|       173|d2b9f5a4-e457-406...|-47.9247|-38.5258|2023-07-08 06:10:00|
|278b2f82-087a-469...|       128|742ceebe-7cbd-42a...|100.6432| 60.0272|2023-03-01 02:16:00|
|ea0b5fad-ff1d-491...|       148|f079c279-1249-40e...|112.6225| -7.7611|2023-09-17 07:09:00|
|f6f4ad28-30e2-4c8...|       110|b8db20aa-9915-426...|100.6432| 60.0272|2023-05-27 10:45:00|
|e63edcf3-eca8-412...|       170|59b61169-ecde-498...|-78.4163| 22.2258|2023-08-24 06:49:00|
|bab58192-fc85-49c...|        42|8ea971d8-bc3e-44c...|106.3456| 35.3371|2023-05-18 00:35:00|
|cafd29e6-3782-4fa...|       197|f0c4dc60-3cf3-4f9...|-47.9247|-38.5258|2023-05-14 09:20:00|
|ce9ed08c-ccdc-48d...|       198|32bf0c7b-f21a-44b...|-63.2369|-17.405

In [21]:
df_planes = spark.read.parquet("/datalake/clean/planes_de_entrenamiento")
df_planes.printSchema()

root
 |-- id_plan: string (nullable = true)
 |-- id_usuario: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- plan_duracion: float (nullable = true)
 |-- instructions: string (nullable = true)
 |-- objectivo: string (nullable = true)



In [22]:
spark.sql("""
CREATE EXTERNAL TABLE planes_de_entrenamiento (
    id_plan STRING,
    id_usuario STRING,
    tipo STRING,
    plan_duracion FLOAT,
    instructions STRING,
    objectivo STRING
)
STORED AS PARQUET
LOCATION '/datalake/clean/planes_de_entrenamiento';
""")

DataFrame[]

In [23]:
spark.sql("select * from planes_de_entrenamiento").show()

+-------+--------------------+----------------+-------------+--------------------+--------------------+
|id_plan|          id_usuario|            tipo|plan_duracion|        instructions|           objectivo|
+-------+--------------------+----------------+-------------+--------------------+--------------------+
|      1|9efa1289-5256-49c...|   media-maraton|         1.63|Lorem ipsum dolor...|Maecenas leo odio...|
|      2|b4d30551-59cb-45b...|            hiit|         1.73|Nulla ut erat id ...|Duis bibendum. Mo...|
|      3|effbcd55-e394-487...|condicionamiento|          1.3|Proin leo odio po...|Donec diam neque,...|
|      4|fc621f46-e0ef-4ad...|            otro|         1.25|Cum sociis natoqu...|Quisque id justo ...|
|      5|5cc10861-3ddd-467...|            hiit|         1.67|Duis consequat du...|Maecenas leo odio...|
|      6|1ee35bd7-2a17-4a8...|        triatlon|          1.0|Cras non velit ne...|Mauris enim leo, ...|
|      7|969d8cac-57dc-4e6...|            otro|         1.56|Mau

In [24]:
df_planes = spark.read.parquet("/datalake/clean/dispositivos")
df_planes.printSchema()

root
 |-- id_dispositivo: string (nullable = true)
 |-- model: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- operating_system: string (nullable = true)



In [25]:
spark.sql("""
CREATE EXTERNAL TABLE dispositivos (
    id_dispositivo STRING,
    model STRING,
    brand STRING,
    operating_system STRING
)
STORED AS PARQUET
LOCATION '/datalake/clean/dispositivos';
""")

DataFrame[]

In [26]:
spark.sql("select * from dispositivos").show()

+--------------------+----------------+-------+----------------+
|      id_dispositivo|           model|  brand|operating_system|
+--------------------+----------------+-------+----------------+
|5f6742cf-0cc3-499...|   iPhone 12 Pro|  Apple|             iOS|
|9c9d7ede-4915-4fd...|         Pixel 5| Google|         Android|
|af1e4b22-9836-4ff...|    Xperia 1 III|   Sony|         Android|
|5d5aa38b-9e5e-412...|     Find X3 Pro|   Oppo|         Android|
|39e8db1f-0258-459...|          Velvet|     LG|         Android|
|6188ae01-946b-40e...|         X60 Pro|   Vivo|         Android|
|0b8d0acd-7391-449...|           9 Pro|OnePlus|         Android|
|cac63327-f1b8-4ba...|Galaxy S21 Ultra|Samsung|         Android|
|a8ec5dc7-b977-432...|           Mi 11| Xiaomi|         Android|
+--------------------+----------------+-------+----------------+

